In [1]:
import pandas

In [2]:
dataset = pandas.read_csv('books_data.csv')

In [ ]:
dataset.head(50)

In [13]:
def get_year_regex(text):
    matches = []
    if text == "":
        return None
    
    i = 0
    while i < len(text):
        if text[i].isdigit():
            digits = ""
            while i < len(text) and text[i].isdigit() and len(digits) < 4:
                digits += text[i]
                i += 1
            if len(digits) == 4:
                matches.append(int(digits))
        else:
            i += 1
            
    return matches
            

In [17]:
text = "20-25-2021"
print(get_year_regex(text))

[2021]
